# Comparison of McBEF vs Classic Plume Rise Simulations

This notebook compares the resulting Plume Rise simulations.

* Freitas PR model with McBEF flaming heat flux
* Freitas PR model with classical heat flux assumptions implemented in plumerise.bioma

Notice that the classic flaming heatfluxes and the fraction of flaming/smoldering fires are prescribed as a function of biome 

In [1]:
import os

from IPython.display import display, Markdown

import numpy as np
import xarray as xr

from datetime import datetime, timedelta

from pyobs import mcbef as mb

from plumerise import bioma   as bm
from plumerise import freitas as pr

import matplotlib.pyplot as plt
%matplotlib inline  

mode = mb.STATS['mode']


## McBEF Level 2 Data and Sampled Meteorology

In [2]:
def getFiresMetOneDay(day,Verbose=False):
    """
    Ingests McBEF retrievals and sampled meteorology for one day.
    """

    l2_dirn = '/css/viirs/data/Level2/VNP47MCBEF/2019/'
    f_fname = l2_dirn + 'VNP47MCBEF.Stats.A2019%03d.0000_2400.Uniform_v_1_0_0.nc'%day
    m_fname = l2_dirn + 'VNP47MCBEF.Met.A2019%03d.nc'%day
    
    f = mb.open_dataset(f_fname) # fire property statistics
    m = xr.open_dataset(m_fname) # sample met fields
    
    # Only good QA
    # ------------
    N = f.dims['fire']
    I = f.QA_flag<mb.QA['Bowtie']
    J = np.arange(N,dtype=int)[I]
    f = f.isel(fire=J)
    m = m.isel(time=J)
    
    failed = int(0.5 + 100 * (N - f.dims['fire']) / N)
    
    if Verbose:
        display(Markdown("**FIRE PROPERTIES**"),f)
        display(Markdown("**SAMPLED METEOROLOGY**"),m)
    
    return f, m, failed


## Calculate Plume Rise for each day

In [3]:
pr.omp_set_num_threads(46) # number of OMP threads
p_dirn = './'
Subset = False # for debugging purposes
Verb = False   # for debugging purposes

oneday = timedelta(seconds=24*60*60)
date0 = datetime(2018,12,31)

#for day in range(290,305):
#for day in [182]:
#for day in [205, 226, 265, 269, 272, 274, 288, 289]:
#for day in range(182,305): # July-October 2019
for day in range(251,305): # July-October 2019   
    
    # Skip problem days...
    # --------------------
    if day in [205, 226, 265, 269, 272, 274, 288, 289]:
        continue
    
    # Load fires and met fields
    # -------------------------
    f, m, failed = getFiresMetOneDay(day,Verbose=Verb)
 
    date_ = (date0+day*oneday).date()
    if day%4 == 0: print('| Day',date_,'with',f.dims['fire'],'fires(',failed,'% failed)',end='\r')
    if day%4 == 1: print('/ Day',date_,'with',f.dims['fire'],'fires(',failed,'% failed)',end='\r')
    if day%4 == 2: print('- Day',date_,'with',f.dims['fire'],'fires(',failed,'% failed)',end='\r')
    if day%4 == 3: print('\ Day',date_,'with',f.dims['fire'],'fires(',failed,'% failed)',end='\r') 
     
    # Optional subsetting
    # -------------------
    if Subset:
        I = np.arange(3000,dtype=int)
        f = f.isel(fire=I)
        m = m.isel(time=I)
        
    # Area used by entrainment parameterization
    # -----------------------------------------
    HeatFlux_MW = f.HeatFlux_f[:,mode].values   # use mode for now
    #Area_m2 = 0.1 * f.FP_Area.values  # entrainment sensitivity
    Area_m2 = f.Area_f[:,mode] +  f.Area_s[:,mode] # entrainment sensitivity
    #Area_m2 =  20e4 * np.ones(f.dims['fire']) # classic 20 hectare fires
    p_filen = p_dirn+'VNP47MCBEF.PR.A2019%03d.nc'%day
    
    # Calculate plume rise
    # --------------------
    z_i, z_d, z_a, z_f, z_plume, w_plume, rc = pr.getPlumesVMD(HeatFlux_MW, Area_m2, m,  
                                                               getW=True,Verb=Verb) 
    
    # Calculate vertical mass distribution
    # ------------------------------------
    ds = pr.getVMD(z_i, z_d, z_a, z_f, m, z_plume=z_plume, w_plume=w_plume)
    
    # Save plume rise results to a file
    # ---------------------------------
    ds.to_netcdf(p_filen)
    
    if Verb: 
        display(ds)

 Open MP maximum number of threads:           46


## Check for Missing Days

In [6]:
oneday = timedelta(seconds=24*60*60)
date0 = datetime(2018,12,31)
miss_day, miss_date = [], []
for day in range(182,305):
    date_ = (date0+day*oneday).date()
    if not os.path.exists('./VNP47MCBEF.PR.A2019%03d.nc'%day):
        miss_day  += [day,]
        miss_date += [date_,]
        print('- Missing',day,date_)
        
print(miss_day)                        

- Missing 205 2019-07-24
- Missing 226 2019-08-14
- Missing 265 2019-09-22
- Missing 269 2019-09-26
- Missing 272 2019-09-29
- Missing 274 2019-10-01
- Missing 288 2019-10-15
- Missing 289 2019-10-16
[205, 226, 265, 269, 272, 274, 288, 289]


In [5]:
f

<xarray.McBEF_Dataset>
Dimensions:         (fire: 4503, stat: 5)
Coordinates:
    FP_Latitude     (fire) float32 ...
    FP_Longitude    (fire) float32 ...
    FP_Time         (fire) datetime64[ns] ...
Dimensions without coordinates: fire, stat
Data variables: (12/30)
    crs             int64 ...
    t_b             (fire) float32 ...
    C               (fire) float32 ...
    QA_flag         (fire) uint32 2 2 2 2 2 2 2 2 2 2 2 ... 2 2 2 2 2 2 2 2 2 2
    FP_Power_R_AC   (fire) float32 ...
    FP_confidence   (fire) int8 ...
    ...              ...
    t_f             (fire, stat) float32 ...
    t_s             (fire, stat) float32 ...
    p_s             (fire, stat) float32 ...
    p_f             (fire, stat) float32 ...
    Area_s          (fire, stat) float32 ...
    Area_f          (fire, stat) float32 ...
Attributes: (12/40)
    history:             Created Sun Nov 17 14:14:30 2024 main_reprocessing.py
    LongName:            MCBEF fire parameter estimation on FILDA VNP47MOD.
    satellite:           VNP
    sel_bg_bands:        ['M14', 'M15', 'I05', 'M16']
    sel_fire_bands:      ['DNB', 'M11', 'M13', 'M14', 'M15', 'M16']
    num_draw:            500
    ...                  ...
    sigma_C:             0.5
    thd_frp:             0.0
    frp_sigma_scale:     0.2
    SNR_bg:              {'DNB': 6.0, 'M08': 20.0, 'M10': 20.0, 'M11': 30.0, ...
    SNR_fire:            {'DNB': 6.0, 'M08': 20.0, 'M10': 40.0, 'M11': 60.0, ...
    precompile_string:   ,compiledir_mode=readonly

In [7]:
ds

<xarray.Dataset>
Dimensions:  (time: 4503, lev: 42, z_plume: 200)
Coordinates:
  * time     (time) datetime64[ns] 2019-10-31 2019-10-31 ... 2019-10-31T23:54:00
  * lev      (lev) float64 31.0 32.0 33.0 34.0 35.0 ... 68.0 69.0 70.0 71.0 72.0
  * z_plume  (z_plume) float32 0.0 100.0 200.0 ... 1.97e+04 1.98e+04 1.99e+04
    lon      (time) float32 ...
    lat      (time) float32 ...
Data variables:
    z_i      (time) float32 1.4e+03 1.4e+03 1.3e+03 900.0 ... 900.0 900.0 900.0
    z_d      (time) float32 1.7e+03 1.7e+03 1.7e+03 ... 900.0 900.0 1.7e+03
    z_a      (time) float32 1.788e+03 1.78e+03 1.783e+03 ... 1.39e+03 1.687e+03
    z_f      (time) float32 2.1e+03 2.1e+03 2.1e+03 ... 2.2e+03 2e+03 2.4e+03
    z_c      (time) float32 1.75e+03 1.75e+03 1.7e+03 ... 1.45e+03 1.65e+03
    delta    (time) float32 350.0 350.0 400.0 550.0 ... 550.0 650.0 550.0 750.0
    vmd      (time, lev) float32 0.0 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0 0.0
    z        (time, lev) float32 2.299e+04 2.192e+04 2.085e+04 ... 201.3 66.78
    zs       (time) float32 205.7 194.2 162.7 ... 1.423e+03 1.423e+03 38.38
    w_plume  (time, z_plume) float32 10.17 6.656 5.426 4.461 ... 0.0 0.0 0.0 0.0
Attributes:
    option:   centered